[코사인유사도 기반 와인 추천시스템 개발]

- 제공받은 csv 파일 로드
- 상위 5,000개의 와인 데이터만 추출하여 변수에 저장
- description 열 추출
- 불용어 제거 (길이가 짧은 단어, 불용어 사전에 등록된 단어, 5,000개 문서 중 단어 빈도 수가 3개 이하인 경우 제거, 등)
- DTM 구성 (if 시간이 많이 소요되면, 상위 1,000개로 구성한다.)
- TF-IDF 행렬 구성 (Matrix Shape : 5,000 * 단어 개수)
- 코사인 유사도 (Matrix Shape : 5,000 * 5,000)
예시) Problem : 번호가 50번에 해당되는 와인과 가장 유사한 와인을 추천해줘 Answer : 가장 유사한 와인 10개 추천
- 대부분 cosine 유사도가 0일텐데, 여기서 유사도가 가장 최댓값인 10개의 와인 추출하기

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df = pd.read_csv("/content/winemag-data-130k-v2.csv")
df

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,NaN,Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
129969,129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


In [ ]:
df.shape

(129971, 14)

In [ ]:
#상위 5,000개 데이터 추출
df = df.head(5000)

In [ ]:
df

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4995,New Zealand,"Sweaty and pungent, loaded with overtones of c...",Swan,88,19.0,Marlborough,NaN,NaN,Joe Czerwinski,@JoeCz,Mud House 2007 Swan Sauvignon Blanc (Marlborough),Sauvignon Blanc,Mud House
4996,4996,Italy,Here's a well-toasted red wine from Southern I...,Cunto,88,45.0,Southern Italy,Campania,NaN,NaN,NaN,Fattoria Alois 2006 Cunto Pallagrello (Campania),Pallagrello,Fattoria Alois
4997,4997,Italy,Candied fruit and caramel ride on the back of ...,Fine Sweet,88,12.0,Sicily & Sardinia,Marsala,NaN,NaN,NaN,Florio NV Fine Sweet (Marsala),Catarratto,Florio
4998,4998,US,This Cabernet Sauvignon-Syrah blend is soft an...,Le Petit Vice,88,50.0,California,Sonoma-Napa,Napa-Sonoma,NaN,NaN,Vice Versa 2005 Le Petit Vice Cabernet Sauvign...,Cabernet Sauvignon-Syrah,Vice Versa


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             5000 non-null   int64  
 1   country                4997 non-null   object 
 2   description            5000 non-null   object 
 3   designation            3523 non-null   object 
 4   points                 5000 non-null   int64  
 5   price                  4657 non-null   float64
 6   province               4997 non-null   object 
 7   region_1               4208 non-null   object 
 8   region_2               1956 non-null   object 
 9   taster_name            3969 non-null   object 
 10  taster_twitter_handle  3794 non-null   object 
 11  title                  5000 non-null   object 
 12  variety                5000 non-null   object 
 13  winery                 5000 non-null   object 
dtypes: float64(1), int64(2), object(11)
memory usage: 547.0+

In [ ]:
# 여기서 우리가 필요한건 description, title - null 값 없음
df.isnull().sum(axis = 0)

Unnamed: 0                  0
country                     3
description                 0
designation              1477
points                      0
price                     343
province                    3
region_1                  792
region_2                 3044
taster_name              1031
taster_twitter_handle    1206
title                       0
variety                     0
winery                      0
dtype: int64

In [ ]:
#dtm 구성
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

count_vectorizer = CountVectorizer(token_pattern=r'\b\w{3,}\b', stop_words='english')
dtm_matrix = count_vectorizer.fit_transform(df['description'])

In [ ]:
dtm_matrix.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
# 불용어, 길이가 짧은 (2글자 이하) 경우에는 제거
tfidf = TfidfVectorizer(stop_words = 'english', min_df=3, token_pattern=r'\b\w{3,}\b')

In [ ]:
#단어는 8179 → 8053개, 와인의 종류는 5,000개

tfidf_m = tfidf.fit_transform(df['description'])
tfidf_m.shape

(5000, 3322)

In [ ]:
tfidf_m

<5000x3322 sparse matrix of type '<class 'numpy.float64'>'
	with 108445 stored elements in Compressed Sparse Row format>

In [ ]:
#코사인 유사도 사용
cos_sim = cosine_similarity(tfidf_m, tfidf_m)

In [ ]:
cos_sim.shape

(5000, 5000)

In [ ]:
cos_sim[0]

array([1.        , 0.01272033, 0.01279775, ..., 0.00850713, 0.        ,
       0.03136576])

In [ ]:
df['title']
df.index

RangeIndex(start=0, stop=5000, step=1)

In [ ]:
df['title']

0                       Nicosia 2013 Vulkà Bianco  (Etna)
1           Quinta dos Avidagos 2011 Avidagos Red (Douro)
2           Rainstorm 2013 Pinot Gris (Willamette Valley)
3       St. Julian 2013 Reserve Late Harvest Riesling ...
4       Sweet Cheeks 2012 Vintner's Reserve Wild Child...
                              ...                        
4995    Mud House 2007 Swan Sauvignon Blanc (Marlborough)
4996     Fattoria Alois 2006 Cunto Pallagrello (Campania)
4997                      Florio NV Fine Sweet  (Marsala)
4998    Vice Versa 2005 Le Petit Vice Cabernet Sauvign...
4999    Viña Mar de Casablanca 2008 Reserva Especial S...
Name: title, Length: 5000, dtype: object

In [ ]:
data = dict(zip(df.index, df['title']))
data

{0: 'Nicosia 2013 Vulkà Bianco  (Etna)',
 1: 'Quinta dos Avidagos 2011 Avidagos Red (Douro)',
 2: 'Rainstorm 2013 Pinot Gris (Willamette Valley)',
 3: 'St. Julian 2013 Reserve Late Harvest Riesling (Lake Michigan Shore)',
 4: "Sweet Cheeks 2012 Vintner's Reserve Wild Child Block Pinot Noir (Willamette Valley)",
 5: 'Tandem 2011 Ars In Vitro Tempranillo-Merlot (Navarra)',
 6: 'Terre di Giurfo 2013 Belsito Frappato (Vittoria)',
 7: 'Trimbach 2012 Gewurztraminer (Alsace)',
 8: 'Heinz Eifel 2013 Shine Gewürztraminer (Rheinhessen)',
 9: 'Jean-Baptiste Adam 2012 Les Natures Pinot Gris (Alsace)',
 10: 'Kirkland Signature 2011 Mountain Cuvée Cabernet Sauvignon (Napa Valley)',
 11: 'Leon Beyer 2012 Gewurztraminer (Alsace)',
 12: 'Louis M. Martini 2012 Cabernet Sauvignon (Alexander Valley)',
 13: 'Masseria Setteporte 2012 Rosso  (Etna)',
 14: 'Mirassou 2012 Chardonnay (Central Coast)',
 15: 'Richard Böcking 2013 Devon Riesling (Mosel)',
 16: 'Felix Lavaque 2010 Felix Malbec (Cafayate)',
 17: 'Ga

In [ ]:
cos_sim

array([[1.        , 0.01272033, 0.01279775, ..., 0.00850713, 0.        ,
        0.03136576],
       [0.01272033, 1.        , 0.02308952, ..., 0.00668297, 0.01616887,
        0.        ],
       [0.01279775, 0.02308952, 1.        , ..., 0.00672364, 0.01469438,
        0.03976313],
       ...,
       [0.00850713, 0.00668297, 0.00672364, ..., 1.        , 0.0120116 ,
        0.01478213],
       [0.        , 0.01616887, 0.01469438, ..., 0.0120116 , 1.        ,
        0.0145738 ],
       [0.03136576, 0.        , 0.03976313, ..., 0.01478213, 0.0145738 ,
        1.        ]])

In [ ]:
def recommend(idx, cosine_sim = cos_sim):
  t = data[idx]
  sim_scores = list(enumerate(cosine_sim[idx]))
  ss = sorted(sim_scores, key = lambda x:x[1], reverse = True)
  ss_scores = ss[1:11]
  res = [i[0] for i in ss_scores]
  return df['title'].iloc[res]

In [ ]:
recommend(49)

30      Domaine de la Madone 2012 Nouveau  (Beaujolais...
4091     Casca Wines 2013 Santos da Casa Red (Alentejano)
2396    Rui Roboredo Madeira 2015 Beyra Colheita Red (...
4561    Jose Maria Vieira 2014 Borges Quinta da Soalhe...
3602                    La Croix de Renaud 2009  Bordeaux
4563    Les Vignerons Réunis de Monségur 2014 Château ...
4397    Georges Duboeuf 2015 Clos Reyssie  (Pouilly-Fu...
3400    Parras Vinhos 2014 Montaria Tinto Red (Alentej...
1552    Adega Cooperativa de Borba 2016 Cuvée B Red (A...
4398    Georges Duboeuf 2015 Domaine du Bois Rosier  (...
Name: title, dtype: object